In [1]:
import os
import requests
import json
import pandas as pd
from datetime import datetime
from dotenv import load_dotenv
from requests import Response

os.chdir("..")
load_dotenv()

True

In [2]:
def format_bulk_stocks_eod(ticker: str, df: pd.DataFrame) -> bytes:
    today = datetime.now().strftime('%Y-%m-%d')
    index_name = f"quant-agents_stocks-eod_{today}"
    lines = []

    for _, row in df.iterrows():

        date_reference = row.get('timestamp')
        open_ = row.get('open')
        close = row.get('close')
        high = row.get('high')
        low = row.get('low')
        volume = row.get('volume')

        if open_ is None or close is None:
            continue
        id_str = f"{ticker}_{str(date_reference)}"

        meta = {"index": {"_index": index_name, "_id": id_str}}

        doc = {
            "key_ticker": ticker,
            "date_reference": date_reference,
            "val_open": float(open_),
            "val_close": float(close) if close is not None else None,
            "val_high": float(high) if high is not None else None,
            "val_low": float(low) if low is not None else None,
            "val_volume": int(volume) if volume is not None else None,
        }

        lines.append(json.dumps(meta))
        lines.append(json.dumps(doc))

    return (("\n".join(lines)) + "\n").encode("utf-8")

def ingest_stocks_eod(ticker: str) -> Response:
    es_url = os.environ.get('ELASTICSEARCH_URL')
    es_api_key = os.environ.get('ELASTICSEARCH_API_KEY')
    alpha_vantage_api_key = os.environ.get('ALPHAVANTAGE_API_KEY')
    alpha_vantage_time_series_url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={ticker}&apikey={alpha_vantage_api_key}&datatype=csv"

    ticker_daily_time_series = pd.read_csv(alpha_vantage_time_series_url)

    return requests.post(
        url=f"{es_url}/_bulk",
        headers={
            'Authorization': f'ApiKey {es_api_key}',
            'Content-Type': 'application/x-ndjson'
        },
        data=format_bulk_stocks_eod(ticker, ticker_daily_time_series)
    )


In [3]:
stocks_eod_response = ingest_stocks_eod("IBM")
print(f"Stocks EOD ingestion complete with errors: {stocks_eod_response.json().get('errors')}")

Stocks EOD ingestion complete with errors: False


In [4]:
def format_bulk_stocks_insider_trades(ticker: str, df: pd.DataFrame) -> bytes:
    today = datetime.now().strftime('%Y-%m-%d')
    index_name = f"quant-agents_stocks-insider-trades_{today}"
    lines = []

    for _, row in df.iterrows():

        date_reference = row.get('transaction_date')
        executive = row.get('executive')
        executive_title = row.get('executive_title')
        acquisition_or_disposal = row.get('acquisition_or_disposal')
        shares= row.get('shares')
        share_price = row.get('share_price')

        id_str = f"{ticker}_{str(date_reference)}"

        meta = {"index": {"_index": index_name, "_id": id_str}}

        doc = {
            "key_ticker": ticker,
            "date_reference": date_reference.strftime('%Y-%m-%d'),
            "text_executive_name": executive,
            "text_executive_title": executive_title,
            "key_acquisition_disposal": acquisition_or_disposal,
            "val_share_quantity": float(shares),
            "val_share_price": float(share_price)
        }

        lines.append(json.dumps(meta))
        lines.append(json.dumps(doc))

    return (("\n".join(lines)) + "\n").encode("utf-8")

def ingest_stocks_insider_trades(ticker: str, cutoff_days=100) -> Response:
    es_url = os.environ.get('ELASTICSEARCH_URL')
    es_api_key = os.environ.get('ELASTICSEARCH_API_KEY')
    alpha_vantage_api_key = os.environ.get('ALPHAVANTAGE_API_KEY')
    alpha_vantage_insider_trades_url = f"https://www.alphavantage.co/query?function=INSIDER_TRANSACTIONS&symbol={ticker}&apikey={alpha_vantage_api_key}"

    ticker_insider_trades_data = requests.get(alpha_vantage_insider_trades_url).json()
    df = pd.json_normalize(ticker_insider_trades_data['data'])
    df["transaction_date"] = pd.to_datetime(df["transaction_date"], errors='coerce')
    cutoff = pd.Timestamp.now() - pd.Timedelta(days=cutoff_days)
    ticker_recent_insider_trades = df[df["transaction_date"] >= cutoff].reset_index(drop=True)

    return requests.post(
        url=f"{es_url}/_bulk",
        headers={
            'Authorization': f'ApiKey {es_api_key}',
            'Content-Type': 'application/x-ndjson'
        },
        data=format_bulk_stocks_insider_trades(ticker, ticker_recent_insider_trades)
    )

In [5]:
stocks_insider_trades_response = ingest_stocks_insider_trades("IBM")
print(f"Stocks insider trades ingestion complete with errors: {stocks_insider_trades_response.json().get('errors')}")

Stocks insider trades ingestion complete with errors: False


In [6]:
def format_bulk_stocks_metadata(ticker: str, df: pd.DataFrame) -> bytes:
    import json
    today = datetime.now().strftime('%Y-%m-%d')
    index_name = f"quant-agents_stocks-metadata_{today}"
    lines = []

    for _, row in df.iterrows():
        # canonical values, accept multiple key variants
        symbol = row.get('Symbol') or ticker
        asset_type = row.get('AssetType')
        name = row.get('Name')
        description = row.get('Description')
        cik = row.get('CIK')
        exchange = row.get('Exchange')
        currency = row.get('Currency')
        country = row.get('Country')
        sector = row.get('Sector')
        industry = row.get('Industry')
        address = row.get('Address')
        official_site = row.get('OfficialSite')
        fiscal_year_end = row.get('FiscalYearEnd')

        latest_quarter = row.get('LatestQuarter')
        dividend_date = row.get('DividendDate')
        ex_dividend_date = row.get('ExDividendDate')

        # numeric conversions
        market_capitalization = int(row.get('MarketCapitalization'))
        ebitda = int(row.get('EBITDA'))
        pe_ratio = float(row.get('PERatio'))
        peg_ratio = float(row.get('PEGRatio'))
        book_value = float(row.get('BookValue'))
        dividend_per_share = float(row.get('DividendPerShare'))
        dividend_yield = float(row.get('DividendYield'))
        eps = float(row.get('EPS'))
        revenue_per_share_ttm = float(row.get('RevenuePerShareTTM'))
        profit_margin = float(row.get('ProfitMargin'))
        operating_margin_ttm = float(row.get('OperatingMarginTTM'))
        return_on_assets_ttm = float(row.get('ReturnOnAssetsTTM'))
        return_on_equity_ttm = float(row.get('ReturnOnEquityTTM'))

        revenue_ttm = int(row.get('RevenueTTM'))
        gross_profit_ttm = int(row.get('GrossProfitTTM'))
        diluted_eps_ttm = float(row.get('DilutedEPSTTM'))

        quarterly_earnings_growth_yoy = float(row.get('QuarterlyEarningsGrowthYOY'))
        quarterly_revenue_growth_yoy = float(row.get('QuarterlyRevenueGrowthYOY'))

        analyst_target_price = float(row.get('AnalystTargetPrice'))
        analyst_rating_strong_buy = int(row.get('AnalystRatingStrongBuy'))
        analyst_rating_buy = int(row.get('AnalystRatingBuy'))
        analyst_rating_hold = int(row.get('AnalystRatingHold'))
        analyst_rating_sell = int(row.get('AnalystRatingSell'))
        analyst_rating_strong_sell = int(row.get('AnalystRatingStrongSell'))

        trailing_pe = float(row.get('TrailingPE'))
        forward_pe = float(row.get('ForwardPE'))
        price_to_sales_ratio_ttm = float(row.get('PriceToSalesRatioTTM'))
        price_to_book_ratio = float(row.get('PriceToBookRatio'))
        ev_to_revenue = float(row.get('EVToRevenue'))
        ev_to_ebitda = float(row.get('EVToEBITDA'))
        beta = float(row.get('Beta'))

        week_52_high = float(row.get('52WeekHigh'))
        week_52_low = float(row.get('52WeekLow'))
        moving_average_50_day = float(row.get('50DayMovingAverage'))
        moving_average_200_day = float(row.get('200DayMovingAverage'))

        shares_outstanding = int(row.get('SharesOutstanding'))
        shares_float = int(row.get('SharesFloat'))
        percent_insiders = float(row.get('PercentInsiders'))
        percent_institutions = float(row.get('PercentInstitutions'))

        # build id using symbol and latest_quarter or today
        id_suffix = latest_quarter or today
        id_str = f"{symbol}_{id_suffix}"

        meta = {"index": {"_index": index_name, "_id": id_str}}

        doc = {
            "key_ticker": symbol,
            "asset_type": asset_type,
            "name": name,
            "description": description,
            "cik": cik,
            "exchange": exchange,
            "currency": currency,
            "country": country,
            "sector": sector,
            "industry": industry,
            "address": address,
            "official_site": official_site,
            "fiscal_year_end": fiscal_year_end,
            "latest_quarter": latest_quarter,
            "market_capitalization": market_capitalization,
            "ebitda": ebitda,
            "pe_ratio": pe_ratio,
            "peg_ratio": peg_ratio,
            "book_value": book_value,
            "dividend_per_share": dividend_per_share,
            "dividend_yield": dividend_yield,
            "eps": eps,
            "revenue_per_share_ttm": revenue_per_share_ttm,
            "profit_margin": profit_margin,
            "operating_margin_ttm": operating_margin_ttm,
            "return_on_assets_ttm": return_on_assets_ttm,
            "return_on_equity_ttm": return_on_equity_ttm,
            "revenue_ttm": revenue_ttm,
            "gross_profit_ttm": gross_profit_ttm,
            "diluted_eps_ttm": diluted_eps_ttm,
            "quarterly_earnings_growth_yoy": quarterly_earnings_growth_yoy,
            "quarterly_revenue_growth_yoy": quarterly_revenue_growth_yoy,
            "analyst_target_price": analyst_target_price,
            "analyst_rating_strong_buy": analyst_rating_strong_buy,
            "analyst_rating_buy": analyst_rating_buy,
            "analyst_rating_hold": analyst_rating_hold,
            "analyst_rating_sell": analyst_rating_sell,
            "analyst_rating_strong_sell": analyst_rating_strong_sell,
            "trailing_pe": trailing_pe,
            "forward_pe": forward_pe,
            "price_to_sales_ratio_ttm": price_to_sales_ratio_ttm,
            "price_to_book_ratio": price_to_book_ratio,
            "ev_to_revenue": ev_to_revenue,
            "ev_to_ebitda": ev_to_ebitda,
            "beta": beta,
            "week_52_high": week_52_high,
            "week_52_low": week_52_low,
            "moving_average_50_day": moving_average_50_day,
            "moving_average_200_day": moving_average_200_day,
            "shares_outstanding": shares_outstanding,
            "shares_float": shares_float,
            "percent_insiders": percent_insiders,
            "percent_institutions": percent_institutions,
            "dividend_date": dividend_date,
            "ex_dividend_date": ex_dividend_date,
        }

        lines.append(json.dumps(meta))
        lines.append(json.dumps(doc))

    return (("\n".join(lines)) + "\n").encode("utf-8")

def ingest_stocks_metadata(ticker: str) -> Response:
    es_url = os.environ.get('ELASTICSEARCH_URL')
    es_api_key = os.environ.get('ELASTICSEARCH_API_KEY')
    alpha_vantage_api_key = os.environ.get('ALPHAVANTAGE_API_KEY')
    alpha_vantage_overview_url = f"https://www.alphavantage.co/query?function=OVERVIEW&symbol={ticker}&apikey={alpha_vantage_api_key}"

    ticker_overview_data = requests.get(alpha_vantage_overview_url).json()
    ticker_metadata = pd.json_normalize(ticker_overview_data)

    return requests.post(
        url=f"{es_url}/_bulk",
        headers={
            'Authorization': f'ApiKey {es_api_key}',
            'Content-Type': 'application/x-ndjson'
        },
        data=format_bulk_stocks_metadata(ticker, ticker_metadata)
    )

In [7]:
stocks_metadata_response = ingest_stocks_metadata("IBM")
print(f"Stocks metadata ingestion complete with errors: {stocks_metadata_response.json().get('errors')}")

Stocks metadata ingestion complete with errors: False
